----
----
# **Análisis en los últimos 365 días**
----
----

## Carga de tabla y preparación

In [1]:
import pandas as pd
import plotly.graph_objects as go
import datetime as dt

In [2]:
# Importamos tabla principal
table = pd.read_csv('my_tables/table_hist.csv')
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       7468 non-null   object 
 1   dia_semana  7468 non-null   object 
 2   semana      7468 non-null   object 
 3   dolar_b     7468 non-null   float64
 4   dolar_o     7468 non-null   float64
 5   brecha_s    7468 non-null   float64
 6   brecha      7468 non-null   float64
 7   db_var      7468 non-null   float64
 8   do_var      7468 non-null   float64
 9   brecha_var  7468 non-null   float64
 10  evento_p    36 non-null     object 
 11  evento_t    36 non-null     object 
dtypes: float64(7), object(5)
memory usage: 700.2+ KB


1) Veo que la fecha no está guardada como datetime, por lo que procedo a convertirla
2) Hay nulos en eventos, por lo que los cambio por strings vacíos.

In [3]:
table['fecha'] = pd.to_datetime(table['fecha'])
table['evento_p'] = table['evento_p'].fillna('') 
table['evento_t'] = table['evento_t'].fillna('')
table.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       7468 non-null   datetime64[ns]
 1   dia_semana  7468 non-null   object        
 2   semana      7468 non-null   object        
 3   dolar_b     7468 non-null   float64       
 4   dolar_o     7468 non-null   float64       
 5   brecha_s    7468 non-null   float64       
 6   brecha      7468 non-null   float64       
 7   db_var      7468 non-null   float64       
 8   do_var      7468 non-null   float64       
 9   brecha_var  7468 non-null   float64       
 10  evento_p    7468 non-null   object        
 11  evento_t    7468 non-null   object        
dtypes: datetime64[ns](1), float64(7), object(4)
memory usage: 700.2+ KB


Utilizo sólo los últimos 365 días:

In [4]:
primer_dia = dt.datetime.today() - dt.timedelta(days=366)
table_365 = table[table['fecha'] > primer_dia].copy()
table_365.reset_index(inplace= True, drop=True)
table_365.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       365 non-null    datetime64[ns]
 1   dia_semana  365 non-null    object        
 2   semana      365 non-null    object        
 3   dolar_b     365 non-null    float64       
 4   dolar_o     365 non-null    float64       
 5   brecha_s    365 non-null    float64       
 6   brecha      365 non-null    float64       
 7   db_var      365 non-null    float64       
 8   do_var      365 non-null    float64       
 9   brecha_var  365 non-null    float64       
 10  evento_p    365 non-null    object        
 11  evento_t    365 non-null    object        
dtypes: datetime64[ns](1), float64(7), object(4)
memory usage: 34.3+ KB


----
## 1- Día con mayor variación en la brecha
El cambio en la brecha puede ser tanto positivo como negativo. Lo más apropiado es utilizar un absoluto del cambio:

In [5]:
table_365['brecha_var_abs'] = table_365['brecha_var'].abs()

Grafica de la variación absoluta y la variación

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha_var'], mode='lines', name='Variación'))
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha_var_abs'], mode='lines', name='Variación absoluta',  line=dict(dash='dot')))
fig.update_layout(
    title='Variación de la brecha',
    xaxis_title = 'Fecha',
    yaxis_title = 'Variación del porcentaje')

Obtengo la fecha máxima:

In [7]:
top_1_brech_var = table_365.sort_values(['brecha_var_abs'], ascending=False)[['fecha', 'brecha_var', 'brecha_var_abs', 'evento_t']].head(1)
top_1_brech_var.reset_index(drop=True, inplace = True)
top_1_brech_var

,fecha,brecha_var,brecha_var_abs,evento_t
0,2022-07-21,15.015052,15.015052,


In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha_var'], mode='lines', name='Variación'))
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha_var_abs'], mode='lines', name='Variación absoluta',  line=dict(dash='dot')))
fig.add_trace(go.Scatter(x = top_1_brech_var['fecha'], y = top_1_brech_var['brecha_var'], mode='markers', name='Top variación de brecha'))
fig.update_layout(
    title='Variación de la brecha',
    xaxis_title = 'Fecha',
    yaxis_title = 'Variación del porcentaje')

El día con **mayor variación en la brecha** (tanto normal como absoluta) fue el día **21 de Julio del 2021** con un cambio en la brecha porcentual de **15.01 puntos**

----
## 2- Top 5 días con mayor variación en el dólar

Lo mismo sucede con este punto... Utilizaremos una variación absoluta para cada uno de los dólares

In [9]:
table_365['db_var_abs'] = table_365['db_var'].abs()
table_365['do_var_abs'] = table_365['do_var'].abs()
table_365.head()

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t,brecha_var_abs,db_var_abs,do_var_abs
0,2021-08-05,Thursday,2021-31,180.500000,96.88,83.620000,86.312964,0.000000,0.041305,-0.076957,,,0.076957,0.000000,0.041305
1,2021-08-06,Friday,2021-31,178.500000,96.91,81.590000,84.191518,-1.108033,0.030966,-2.121447,,,2.121447,1.108033,0.030966
2,2021-08-07,Saturday,2021-31,178.666667,96.94,81.726667,84.306444,0.093371,0.030957,0.114926,,,0.114926,0.093371,0.030957
3,2021-08-08,Sunday,2021-31,178.833333,96.97,81.863333,84.421299,0.093284,0.030947,0.114855,,,0.114855,0.093284,0.030947
4,2021-08-09,Monday,2021-32,179.000000,97.00,82.000000,84.536082,0.093197,0.030937,0.114784,,,0.114784,0.093197,0.030937


In [17]:
# Top 5 variaciones absolutas del dólar
top_5_var_db = table_365.sort_values(['db_var_abs'], ascending=False)[['fecha', 'dolar_b', 'db_var', 'db_var_abs', 'evento_t', 'evento_p']].head(5)
top_5_var_do = table_365.sort_values(['do_var_abs'], ascending=False)[['fecha', 'dolar_o', 'do_var', 'do_var_abs', 'evento_t', 'evento_p']].head(5)

**Dólar Blue**

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['db_var'], mode='lines', name='Variación del dólar'))
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['db_var_abs'], mode='lines', name='Variación absoluta del dólar',  line=dict(dash='dot')))
fig.add_trace(go.Scatter(x = top_5_var_db['fecha'], y = top_5_var_db['db_var_abs'], mode='markers', name='Top variación absoluta del dólar'))
fig.update_layout(
    title='Variación del Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje')

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['dolar_b'], mode='lines', name='Valor del Dólar Blue'))
fig.add_trace(go.Scatter(x = top_5_var_db['fecha'], y = top_5_var_db['dolar_b'], mode='markers', name='Top 5 variación absoluta del dólar'))
fig.update_layout(
    title='Dólar Blue',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

**El top 5 fechas de variación del dólar blue son las siguientes:**

In [20]:
top_5_var_db

,fecha,dolar_b,db_var,db_var_abs,evento_t,evento_p
350,2022-07-21,337.0,6.309148,6.309148,,
337,2022-07-08,273.0,6.225681,6.225681,,
358,2022-07-29,296.0,-5.732484,5.732484,,
349,2022-07-20,317.0,5.315615,5.315615,,
176,2022-01-28,212.5,-4.494382,4.494382,,


4 del Top 5 variaciones absolutas del dólar blue fueron en **Julio del 2022**. Tres de las 5 corresponden a variaciones positivas.

**Dólar oficial**

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['do_var'], mode='lines', name='Variación del dólar'))
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['do_var_abs'], mode='lines', name='Variación absoluta del dólar',  line=dict(dash='dot')))
fig.add_trace(go.Scatter(x = top_5_var_do['fecha'], y = top_5_var_do['do_var_abs'], mode='markers', name='Top variación absoluta del dólar'))
fig.update_layout(
    title='Variación del Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Porcentaje')

In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['dolar_o'], mode='lines', name='Valor del Dólar Oficial'))
fig.add_trace(go.Scatter(x = top_5_var_do['fecha'], y = top_5_var_do['dolar_o'], mode='markers', name='Top 5 variación absoluta del dólar'))
fig.update_layout(
    title='Dólar Oficial',
    xaxis_title = 'Fecha',
    yaxis_title = 'Soles')

**El top 5 fechas de variación del dólar oficial son las siguientes:**


In [23]:
top_5_var_do

,fecha,dolar_o,do_var,do_var_abs,evento_t,evento_p
356,2022-07-27,130.89,0.206706,0.206706,,
349,2022-07-20,129.39,0.201347,0.201347,,
323,2022-06-24,124.26,0.185439,0.185439,,
357,2022-07-28,131.13,0.183360,0.183360,econ,Sergio Massa
362,2022-08-02,132.18,0.181901,0.181901,,


Tres del top 5 se encuentran en **Julio del 2022**. La variación top 4 coincide con el nombramiento de Sergio Massa como mínistro de Economía

----
## 3- Semana con mayor variación en la brecha
Agrupamos por semana usando media para la brecha y la variación absoluta

In [30]:
top_week_brecha_var = table_365[['semana', 'brecha_var_abs', 'brecha']].groupby('semana').mean()
top_week_brecha_var.reset_index(inplace = True)
top_week_brecha_var = top_week_brecha_var.sort_values('brecha_var_abs', ascending=False).head(1)
top_week_brecha_var.reset_index(inplace = True, drop=True)
top_week_brecha_var

,semana,brecha_var_abs,brecha
0,2022-29,6.383065,147.389435


Así obtenemos que la semana 29 del año 2022 tiene en promedio la mayor variación en la brecha.<br>
Para visualizar los días:

In [41]:
days = table_365[table_365['semana'] == top_week_brecha_var.loc[0,'semana']][['fecha', 'semana', 'brecha', 'brecha_var', 'brecha_var_abs','evento_p', 'evento_t']]
days.reset_index(drop = True, inplace = True)
days

,fecha,semana,brecha,brecha_var,brecha_var_abs,evento_p,evento_t
0,2022-07-18,2022-29,125.756400,-0.909921,0.909921,,
1,2022-07-19,2022-29,133.098428,7.342028,7.342028,,
2,2022-07-20,2022-29,144.995749,11.897321,11.897321,,
3,2022-07-21,2022-29,160.010802,15.015052,15.015052,,
4,2022-07-22,2022-29,160.480888,0.470086,0.470086,,
5,2022-07-23,2022-29,155.949938,-4.530949,4.530949,,
6,2022-07-24,2022-29,151.433839,-4.516100,4.516100,,


In [42]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha'], mode='lines', name='Brecha'))
fig.add_trace(go.Scatter(x = days['fecha'], y = days['brecha'], mode='lines', name='Top semana de variación en la brecha'))
fig.update_layout(
    title='Brecha',
    xaxis_title = 'Fecha',
    yaxis_title = 'Puntos porcentuales')

In [40]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = table_365['fecha'], y = table_365['brecha_var_abs'], mode='lines', name='Variación absoluta deBrecha'))
fig.add_trace(go.Scatter(x = days['fecha'], y = days['brecha_var_abs'], mode='lines', name='Top semana de variación en la brecha'))
fig.update_layout(
    title='Variación absoluta de la Brecha',
    xaxis_title = 'Fecha',
    yaxis_title = 'Puntos porcentuales')

La semana con mayor promedio en la variación absoluta de la brecha fue la **semana del 18 de Julio de 2022** con un promedio de **6.38%**. <br>
No hay eventos destacables en la semana <br>

----
## 5- Día de la semana donde hay mayor variación en la brecha 
Agrupo por día de la semana y obtengo el promedio de la variación absoluta en la brecha

In [49]:
days_of_week = table_365[['dia_semana', 'brecha', 'brecha_var']].groupby('dia_semana').mean()
days_of_week.sort_values('brecha_var', inplace = True, ascending=False)
days_of_week.reset_index(inplace = True)
days_of_week

,dia_semana,brecha,brecha_var
0,Wednesday,93.013229,0.896068
1,Tuesday,92.117161,0.877584
2,Thursday,92.873195,-0.015066
3,Monday,91.239577,-0.269942
4,Sunday,91.509519,-0.270495
5,Saturday,91.780013,-0.271050
6,Friday,92.051063,-0.312178


In [50]:
days_of_week_abs = table_365[['dia_semana', 'brecha', 'brecha_var_abs']].groupby('dia_semana').mean()
days_of_week_abs.sort_values('brecha_var_abs', inplace = True, ascending=False)
days_of_week_abs.reset_index(inplace = True)
days_of_week_abs

,dia_semana,brecha,brecha_var_abs
0,Wednesday,93.013229,1.864806
1,Friday,92.051063,1.811780
2,Tuesday,92.117161,1.579205
3,Thursday,92.873195,1.456639
4,Saturday,91.780013,0.661811
5,Sunday,91.509519,0.660181
6,Monday,91.239577,0.658558


Tanto en la variación como en la variación absoluta de la brecha obtuvimos que **el día de la semana con mayor promedio de variación es el Miércoles**.